In [1]:
#imports
import numpy as np
import pandas as pd
import math
import pickle

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error

from sklearn.preprocessing import PolynomialFeatures,OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

#import training dataset
train_df = pd.read_csv("samletdata.csv", sep = ';')

#see the columns in our data
train_df.info()

# take a look at the head of the dataset
train_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99187 entries, 0 to 99186
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Unnamed: 0    99187 non-null  int64 
 1   model         99187 non-null  object
 2   year          99187 non-null  int64 
 3   price         99187 non-null  int64 
 4   transmission  99187 non-null  object
 5   mileage       99187 non-null  int64 
 6   fuelType      99187 non-null  object
 7   tax           99187 non-null  int64 
 8   mpg           99187 non-null  object
 9   engineSize    99187 non-null  object
 10  brand         99187 non-null  object
dtypes: int64(5), object(6)
memory usage: 8.3+ MB


,Unnamed: 0,model,year,price,transmission,mileage,fuelType,tax,mpg,engineSize,brand
0,0,A1,2017,12500,Manual,15735,Petrol,150,"55,4","1,4",audi
1,1,A6,2016,16500,Automatic,36203,Diesel,20,"64,2","2,0",audi
2,2,A1,2016,11000,Manual,29946,Petrol,30,"55,4","1,4",audi
3,3,A4,2017,16800,Automatic,25952,Diesel,145,"67,3","2,0",audi
4,4,A3,2019,17300,Manual,1998,Petrol,145,"49,6","1,0",audi


In [2]:
features = ['model', 'year', 'transmission', 'mileage','fuelType', 'mpg', 'engineSize', 'brand']

x = pd.get_dummies(train_df[features])
print(x)

y = train_df['price']

x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=42, test_size=0.3)

       year  mileage  model_ 1 Series  model_ 2 Series  model_ 3 Series   
0      2017    15735            False            False            False  \
1      2016    36203            False            False            False   
2      2016    29946            False            False            False   
3      2017    25952            False            False            False   
4      2019     1998            False            False            False   
...     ...      ...              ...              ...              ...   
99182  2012    74000            False            False            False   
99183  2008    88102            False            False            False   
99184  2009    70000            False            False            False   
99185  2006    82704            False            False            False   
99186  2007    74000            False            False            False   

       model_ 4 Series  model_ 5 Series  model_ 6 Series  model_ 7 Series   
0                False

In [3]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)

print('Training score: {}'.format(lr_model.score(x_train, y_train)))
print('Test score: {}'.format(lr_model.score(x_test, y_test)))

y_pred = lr_model.predict(x_test)
mse = mean_squared_error(y_test, y_pred)
rmse = math.sqrt(mse)

print('RMSE: {}'.format(rmse))

import pickle
pickle.dump(lr_model, open("lr_modeldinmor🤓.pickle", "ab"))

Training score: 0.908602849558245
Test score: 0.9006692273073217
RMSE: 3110.347702695277


In [4]:
steps = [
    ('scalar', StandardScaler(with_mean=False)),
    ('poly', PolynomialFeatures(degree=2)),
    ('model', LinearRegression())
]

pipeline = Pipeline(steps)

pipeline.fit(x_train, y_train)

print('Training score: {}'.format(pipeline.score(x_train, y_train)))
print('Test score: {}'.format(pipeline.score(x_test, y_test)))

MemoryError: Unable to allocate 55.8 GiB for an array with shape (69430, 107880) and data type float64